Analyse des données

Préparation des données avec pandas


In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import polars as pl
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x :.3f}')
from cleaner import DataCleaner
import numpy as np
from datetime import datetime
from tools import (
    logging
)

Dataset name_basics

In [2]:
df = pd.read_csv("clean_datasets/movies.csv", index_col=0)

In [3]:
cleaning = DataCleaner()
# d'abord je fix l'encodage et apres je clean les \N

test1 = cleaning.fix_values(df, "fix_encode")
test = cleaning.fix_values(test1, "fix_n")
# maintenant je clean les porns ! et supprime les ligne ou nous n'avons pas de type de genre.
clean_porn = cleaning.clean_porn(test)

logging.info(f"Cleaned : {len(df) - len(clean_porn)} rows")

2023-10-23 08:49:28 INFO     Fixing encoding values...
2023-10-23 08:49:40 INFO     Fixing N values...
2023-10-23 08:49:44 INFO     Cleaning porn movies...
2023-10-23 08:49:46 INFO     Cleaned : 477069 rows


In [4]:
condi = (
    clean_porn["person_job"].str.contains("act")
)
df = clean_porn[condi]
actor = df.copy()

In [5]:
"""
Nettoyage des date_sorties nan et modification de la colonne.
Nous avons plusieurs methodes disponible, comme:

clean_porn['titre_date_sortie'].fillna(-1, inplace=True)
clean_porn.dropna(subset=['titre_date_sortie'], inplace=True)
et une fois que l'on a clean
on applique int a toute la colonne
clean_porn['titre_date_sortie'] = clean_porn['titre_date_sortie'].astype(int)

"""
actor['titre_date_sortie'].fillna(0, inplace=True)
actor['titre_date_sortie'] = actor['titre_date_sortie'].astype("int64")
actor['person_birthdate'] = actor['person_birthdate'].astype("int64")
actor['titre_duree'] = actor['titre_duree'].astype("int64")

line = "clean_datasets/movies_rating.csv"
actor.to_csv(line)

In [ ]:
actor.tail()

In [ ]:
"""
La moyenne d'age des acteurs a la date de sortie du film.
Nous calculons la mediane sur cette base.

other KPI, calcul la moyenne des ages en fonction de l'année

"""
# ajouter colonne age pour les personnes
actor["person_age"] = np.where(
    (actor["person_birthdate"] > 1900) &
    (actor["titre_date_sortie"] > 1900) &
    (actor["titre_date_sortie"] - actor["person_birthdate"] >= 0),
    (actor["titre_date_sortie"] - actor["person_birthdate"]),
    0
)


In [ ]:
"""
imagine un fonction, qui prendrais en argument une liste de date de
titre de sortie et retourne la moyenne d'age de ces années des acteurs.

# Quels sont les acteurs les plus présents ?
# À quelle période ?
La durée moyenne des films s'allonge ou se raccourcit avec les années ?
Les acteurs de série sont-ils les mêmes qu'au cinéma ?
# Les acteurs ont en moyenne quel âge ?
Quels sont les films les mieux notés ?
Partagent-ils des caractéristiques communes ? etc...

"""
dates = ["<1900", ">1900", ">1930", ">1960", ">1990", ">2010"]

actor["cuts"] = pd.cut(
    actor["titre_date_sortie"],
    bins=[0, 1900, 1931, 1961, 1991, 2011, 2030],
    labels=dates
)

for date in dates[1:]:
    condi = (
        (actor["person_age"] != 0) &
        (actor["cuts"] == date)
    )
    print(f"Mediane for {date}", actor["person_age"][condi].median())


In [ ]:
# Top 5 des acteurs all time !
print(actor["person_name"].value_counts().head(5))

In [ ]:
# Top 5 actors by years
for date in dates[1:]:
    top5 = actor['person_name'][condi].value_counts().head(5)
    condi = (
        (actor["cuts"] == date)
    )
    top5_str = '\n'.join(str(top5).split('\n')[:-1])

    print(f"Top 5 actors by movies for {date}\n{'-'*20}\n{top5_str}")
    print("-"*20)


In [ ]:
#  World record of the longest documentary
# source :https://fr.wikipedia.org/wiki/Ambianc%C3%A9
longest = actor[actor["titre_duree"] == 43200]
n = longest[["titre_str", "titre_duree"]].iloc[0]
heure = n["titre_duree"] // 60
jours = heure // 24
print(f"Le film le plus long est {n['titre_str']} d'une durée de {heure} heures soit {jours} jours")
# All time median for duration
print(f"All time duration median : {round(actor['titre_duree'].median())} minutes")


In [ ]:
# Median duration for movies between years.
for date in dates[1:]:
    condi = (
        (actor["cuts"] == date)
    )
    med = round(actor["titre_duree"][condi].median())
    print(f"Median durations for movies {date}\nMedian for this years is {med} minutes")
    print("-"*20)
